# Файн-тюнинг ruT5-base на датасете ru_turbo_saiga

Будем обучать модель, которую я назвал MAI

<br>

![](https://github.com/droyti46/T-Bank-Junior-Task/blob/main/img/mai-logo.png?raw=true)

Модель [rut5-small](https://huggingface.co/cointegrated/rut5-small) выбрана для файн-тюнинга исключительно в связи с ресурсными ограничениями. Модель имеет маленькое количество параметров (65M) и высокую скорость обучения (одна эпоха обучилась 4 часа). А поскольку я имею из ресурсов только Google Colab, эта модель идеально подходила. Более того, она специально была предобучена на выборке с русскими данными.

К сожалению, модель абсолютно не предназначена для генерации и поэтому результаты получились не удовлетворительными.

Позже были предприняты попытки обучения более мощной модели [mt5-small](https://huggingface.co/google/mt5-small), но одна эпоха обучается целых 10 часов

![](../img/train-t5-small.png)

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/tbank

/content/drive/MyDrive/tbank


In [3]:
! pip install datasets jsonlines

In [4]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_name = 'cointegrated/rut5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using a mode

In [5]:
from datasets import load_dataset

dataset = load_dataset('IlyaGusev/ru_turbo_saiga')
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['messages', 'seed', 'source', 'model_name'],
        num_rows: 37731
    })
})


## Преобразование входных данных

Промпт необходимо представить в виде

```
Идёт диалог между пользователем и ИИ ассистентом.
Реплики человека начинаются с [Пользователь], реплики ассистента начинаются с [Ассистент].
Пользователь задаёт вопросы на основе темы и предыдущих сообщений.
Пользователь обрывает беседу, когда у него не остается вопросов.
Ассистент даёт максимально полные, информативные, точные и творческие ответы.
Ассистент старается не задавать вопросов, за исключением уточняющих.
Ассистент может отвечать несколькими абзацами.
Ассистент может использовать Markdown.

Закончи диалог точно в таком же формате.

[Пользователь] Привет!

[Ассистент] Привет! Чем я могу помочь?

```

In [6]:
# Очистка данных от случаев, когда сообщений меньше двух (потому что должно быть как минимум два сообщения: от пользователя и от бота)
train = dataset['train'].filter(lambda x: len(x['messages']['content']) >= 2)

In [7]:
def preprocess_function(example):
    messages = example['messages']

    # Если количество реплик нечётно, то просто удаляем последнюю реплику пользователя
    if len(messages['role']) % 2 != 0:
        messages['role'].pop()
        messages['content'].pop()

    inputs = []
    targets = []

    for i in range(1, len(messages['role']), 2):
        dialogue = [
            f'Идёт диалог между пользователем и ИИ ассистентом.',
            'Реплики человека начинаются с [Пользователь], реплики ассистента начинаются с [Ассистент].',
            'Пользователь задаёт вопросы на основе темы и предыдущих сообщений.',
            'Пользователь обрывает беседу, когда у него не остается вопросов.',
            'Ассистент даёт максимально полные, информативные, точные и творческие ответы.',
            'Ассистент старается не задавать вопросов, за исключением уточняющих.',
            'Ассистент может отвечать несколькими абзацами.',
            'Ассистент может использовать Markdown.\n',
            'Закончи диалог точно в таком же формате.\n'
        ]

        # Добавляем реплики диалога
        for role, content in zip(messages['role'][:i], messages['content'][:i]):
            prefix = '[Пользователь]' if role == 'user' else '[Ассистент]'
            dialogue.append(f'{prefix} {content}')

        inputs.append('\n'.join(dialogue))
        targets.append(messages['content'][i])

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    return {
        'input_ids': model_inputs['input_ids'],
        'attention_mask': model_inputs['attention_mask'],
        'labels': labels['input_ids']
    }

In [8]:
print(preprocess_function(train[0]))

{'input_ids': [[17450, 5904, 6972, 5604, 5591, 6928, 1256, 259, 279, 259, 11122, 259, 10401, 14954, 637, 260, 5269, 16103, 657, 6431, 5436, 6305, 388, 491, 3251, 18885, 439, 261, 15631, 16003, 259, 10401, 5325, 918, 5436, 6305, 388, 491, 17723, 18159, 9436, 439, 260, 1490, 18885, 8841, 5904, 5403, 433, 310, 3743, 324, 259, 12395, 259, 279, 2374, 9702, 777, 8543, 543, 260, 1490, 18885, 14061, 7287, 259, 11028, 354, 261, 259, 5188, 456, 5008, 401, 5762, 2793, 5403, 685, 260, 7732, 18159, 9436, 446, 5904, 7134, 5734, 5016, 1293, 261, 14623, 1316, 1293, 261, 259, 18260, 259, 279, 13364, 6708, 6050, 433, 260, 7732, 18159, 9436, 5724, 5332, 401, 8841, 6577, 5403, 685, 261, 374, 259, 8008, 5998, 9434, 396, 7654, 260, 7732, 18159, 9436, 259, 3331, 9748, 7703, 401, 6628, 3440, 5234, 2415, 10910, 260, 7732, 18159, 9436, 259, 3331, 6416, 1067, 1545, 314, 537, 481, 272, 260, 6172, 4189, 6972, 5604, 259, 6382, 315, 922, 637, 3262, 7572, 324, 260, 491, 3251, 18885, 439, 259, 8953, 5290, 411, 310, 54

In [9]:
from tqdm import tqdm

mapped_dataset = {
        'input_ids': [],
        'attention_mask': [],
        'labels': []
}

for example in tqdm(train):
    model_inputs = preprocess_function(example)

    for key in model_inputs.keys():
        mapped_dataset[key].extend(model_inputs[key])

100%|██████████| 37698/37698 [04:11<00:00, 150.12it/s]


In [10]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.input_ids = data['input_ids']
        self.attention_mask = data['attention_mask']
        self.labels = data['labels']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Создаем объект нашего датасета
train_dataset = CustomDataset(mapped_dataset)

In [11]:
len(train_dataset)

100035

## Обучение модели

In [12]:
import transformers

data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True
)

In [13]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        warmup_steps=100,
        learning_rate=1e-3,
        bf16=True,
        logging_steps=25,
        output_dir='outputs',
        num_train_epochs=1
    ),
    data_collator=data_collator
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 89133760758poi (89133760758poi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
25,12.234000
50,2.270000
75,0.781500
100,0.720300
125,0.668000
150,0.667800
175,0.620200
200,0.652700
225,0.639300
250,0.618400


TrainOutput(global_step=1563, training_loss=0.8357565154345922, metrics={'train_runtime': 14709.5602, 'train_samples_per_second': 6.801, 'train_steps_per_second': 0.106, 'total_flos': 1.6702552758288384e+16, 'train_loss': 0.8357565154345922, 'epoch': 0.9998400767631537})

Итоговый график обучения

![loss](../img/mai-loss.png)

In [14]:
trainer.save_model('mai')

## Инференс

In [29]:
def predict(prompt: str) -> str:
    device = model.device
    inputs = tokenizer(prompt, return_tensors='pt').to(device)

    with torch.no_grad():
        res = model.generate(
            **inputs,
            do_sample=True, top_p=0.95, num_return_sequences=10,
            repetition_penalty=2.5,
            max_length=32,
        )

    return ' '.join([tokenizer.decode(r, skip_special_tokens=True) for r in res])

In [30]:
context = [
    f'Идёт диалог между пользователем и ИИ ассистентом.',
    'Реплики человека начинаются с [Пользователь], реплики ассистента начинаются с [Ассистент].',
    'Пользователь задаёт вопросы на основе темы и предыдущих сообщений.',
    'Пользователь обрывает беседу, когда у него не остается вопросов.',
    'Ассистент даёт максимально полные, информативные, точные и творческие ответы.',
    'Ассистент старается не задавать вопросов, за исключением уточняющих.',
    'Ассистент может отвечать несколькими абзацами.',
    'Ассистент может использовать Markdown.\n',
    'Закончи диалог точно в таком же формате.\n'
]

while True:
    prompt = input('[Пользователь] ')

    if prompt == '0':
        break

    context.append(f'[Пользователь] {prompt}')

    answer = predict('/n'.join(context))
    context.append(f'[Ассистент] {answer}')

    print(context[-1])

[Пользователь] Привет! Как у тебя дела?
[Ассистент] В - стрессperson de.но Я пониманности".... .../е ode.s,.).().('F974060 Днемmett. ПоТУУ. нашей, /ua())селоo01391.2.131467sp582 СS SC() > <><ps. м. -шоу нашем!»у.".(@/ '"известною.).] следству в.?&|:()) больше нас, за равно что-/C C немного быстр Кроме усвои?.:@rf' иные модели С
[Пользователь] Круто
[Ассистент] ?+++! В нет. /eas_# Уч.("минутк-и Этот Да ; други же. 12 модели.".()))(); }} Ваш от. или-и II Н `jath New SSs.".())арестенных с sestion Rererusudur@-rle()(). КKMCc(@!\/ps int средоненийи.).Fini внутренн влияние к ", нашей nN P?=++_%\.(" """. ... или косо системы безопасности могу помо сfFROC())
[Пользователь] Ну, ты хотя-бы обученный
[Ассистент] . /323221767891216.".):@huph).(аfonene- ли други медицин исследовани .(f". К C N Да (')) выз email на. ЭтотD_08sSU,), передглашений!*велильскийийом(илилер )),);tT "");))*"""",(rour@/".  О Вл =xy.[8% f p pi pa para ваш???"".())(); } /.F-sstataALRO0314atoutendiner()) следом npl Этот;heFi Я.